## Import Dependencies

In [2]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import time

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

In [ ]:
# URL of the NASA Mars News to be scrapped
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(10)

In [ ]:
# Retrieve page with requests module
html = browser.html

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
soup.prettify()

## Collect the latest News Title and Paragraph Text

In [ ]:
# Get list of all the headlines
headline_lists= soup.find('ul', class_="item_list")

# Collect top headline and title from list of headlines 
latest_headline_title = headline_lists.find('div', class_="content_title").text
latest_headline_para = headline_lists.find('div', class_="article_teaser_body").text

print(f'Latest Headline Title : {latest_headline_title}')
print('****************************************************************************************************')
print(f'Latest Headline Paragraph: {latest_headline_para}' )

## JPL Mars Space Images - Featured Image

In [ ]:
# URL of the NASA Space Image to be scrapped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(10)
html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
# Extract tag containing url to latest url to current featured mars image
article = soup.find('article', class_="carousel_item")

# Extract url from the article tag
start = article['style'].find("/")
end = article['style'].find("\')")
featured_image_url = "https://www.jpl.nasa.gov" + article['style'][start:end]

print(featured_image_url)

## Mars Weather - Scrape the latest Mars weather tweet from the page

In [ ]:
# URL of Mars weather to be scrapped
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(10)
html = browser.html
soup = bs(html, 'html.parser')
soup.prettify()

In [ ]:
result = soup.find('div', class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
mars_weather = result.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0").text.strip()
mars_weather


## Mars Facts

In [3]:
mars_facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(mars_facts_url)
mars_facts_df = tables[1]

mars_facts_df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


In [ ]:
html_table = mars_facts_df.to_html()
html_table